In [1]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-uu2z4zdl
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-uu2z4zdl
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.7 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369489 sha256=895fc4a12d50b03adf43d8b76d5dfee53fff123b0ac4e6d48fb94db8f70c5c29
  Stored in directory: /tmp/pip-ephem-wheel-cache-4fo24bxs/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip


In [2]:
from PIL import Image
import numpy as np
import torch
import tensorflow as tf
import clip

In [4]:
interpreter_img = tf.lite.Interpreter(model_path='openai_clip-clipimageencoder.tflite')
interpreter_img.allocate_tensors()


interpreter_txt = tf.lite.Interpreter(model_path='openai_clip-cliptextencoder.tflite')
interpreter_txt.allocate_tensors()

In [11]:
def inference_model(interpreter, input_data):
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    # Prepare input data
    input_data = input_data.cpu().detach().numpy().reshape(input_details[0]['shape'])
    interpreter.set_tensor(input_details[0]['index'], input_data)

    # Run inference
    interpreter.invoke()

    # Get the output
    output_data = interpreter.get_tensor(output_details[0]['index'])

    return output_data

In [8]:
_, preprocess = clip.load("ViT-B/32")

image = preprocess(Image.open("a_photo_of_a_television_studio.png")).unsqueeze(0)
text_list = ["a photo of a cat", "a photo of a dog", "a photo of a television studio", "a photo of a flying cat"]

In [10]:
image_features = inference_model(interpreter_img, image)

for text in text_list:
    text_tokenized = clip.tokenize(text)
    text_features = inference_model(interpreter_txt, text_tokenized)
    # logits_per_image = image_features @ text_features.t()
    prob = np.matmul(image_features, text_features.T)
    print("prob for", text, " :", prob)

prob for a photo of a cat  : [[19.500679]]
prob for a photo of a dog  : [[20.169495]]
prob for a photo of a television studio  : [[29.361773]]
prob for a photo of a flying cat  : [[19.22459]]
